In [ ]:
import concurrent.futures
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import duckdb
from pathlib import Path
import json

# Settings
output_dir = r'output'

In [ ]:
# Load
items = pd.read_parquet(output_dir + "/items.parquet")
# equip = pd.read_parquet(output_dir + "/jsonequips_long.parquet")
item_qualities = pd.read_csv(output_dir + "/item_qualities.csv", sep=";")
item_qualities_palette = sns.color_palette(item_qualities["color"])

In [ ]:
item_equip_cache = {}


def load_item_equip(item_id):
    filepath = Path(output_dir + f"/jsonequip/{item_id}.equip.json")
    if filepath.exists():
        item_equip_cache[item_id] = json.loads(filepath.read_text())
        item_equip_cache[item_id].pop("appearances", None)
        return item_equip_cache[item_id]
    else:
        return None


def get_item_eqiup(item_id):
    cached_val = item_equip_cache.get(item_id, None)
    if cached_val != None:
        return cached_val
    else:
        return load_item_equip(item_id)

In [ ]:
def get_weapon_dps(item_id):
    item_eqiup = get_item_eqiup(item_id)
    return item_eqiup.get('dps') or item_eqiup.get('mledps') or item_eqiup.get('rgddps') or pd.NA

weapons = items[items['class_name'] == 'Weapons'][['subclass_name','name', 'level','quality_id','quality_name']].reset_index()
weapons.rename(columns={'subclass_name':'subclass', 'quality_name':'quality', 'level':'item level', 'id':'item_id'}, inplace=True)

weapons['quality_color'] = pd.Series([(lambda qid: item_qualities_palette[qid])(qid) for qid in weapons['quality_id']])
weapons['dps'] = pd.Series([get_weapon_dps(item_id) for item_id in weapons['item_id']])
weapons = weapons[weapons['dps'].notna()].sort_values('item_id', ascending=True).reset_index().drop(columns='index')
weapons

In [ ]:
sns.set_theme(rc={"figure.figsize": (16, 9)})


def get_weapon_graph(subclass, quality_id=-1):
    data_base = weapons[weapons["subclass"] == subclass]
    title = subclass
    if quality_id >= 0:
        data_base = data_base[data_base["quality_id"] == quality_id]
        title += f" ({item_qualities['name'][quality_id]})"
    # return data_base
    grp = data_base.groupby("item level")
    data = pd.DataFrame(
        {
            "dps": grp["dps"].mean(),
            "quality_color": grp["quality_color"].first(),
        }
    ).sort_index().reset_index()
    #return data
    return sns.lineplot(
        data=data, x="item level", y="dps", hue="quality_color", legend=None#, sort=True
    ).set_title(title)

get_weapon_graph("Daggers", 4)

In [ ]:
data_base = get_weapon_graph("Daggers", 4)
pd.DataFrame(data_base.groupby("item level")[["item level", "quality_color", "dps"]])
grp = data_base.groupby("item level")
pd.DataFrame({'quality_color': grp['quality_color'], 'mean dps': grp['dps'].mean()})